## packages

In [5]:
!pip install geopandas
import geopandas as gpd
import pandas as pd
import json
!pip install -q --upgrade git+https://github.com/SafeGraphInc/safegraph_py
from safegraph_py_functions import safegraph_py_functions as sgpy
from safegraph_py_functions import cbg_functions as sgpy_cbg
from google.colab import drive
import os


In [3]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [6]:
# Set working directory: change as appropriate
#os.chdir('/content/gdrive/My Drive/your path')


## filter raw clothing data

The following function reads all raw visit data files. The raw data is on poi (points of interest, i.e., a location) level, with a json column named "visitor_home_cbgs" storing visitor information.

In [15]:
# you need to have raw visit data stored in the path
# the data used in this study was accessed on April 6, 2023.
# The data structure may have been updated since then. Please verify the data
# structure before using the code provided below.
def single_month_raw_data(month):
  df_poi_monthly = pd.DataFrame()
  # there are 30 sub files for each month
  for i in range(1, 30):
    file_name = f"/yourpath/raw_data/safegraph_{month}_part{i}.csv.gz"
    df_part = pd.read_csv(file_name,compression='zip',usecols=[0,4,6,7,9,10,28,29,30,35,36])
    # filter data by poi category and country
    df_part = df_part[(df_part.top_category == "Clothing Stores") & (df_part.iso_country_code == "US")]
    df_part['visitor_home_cbgs'] = df_part['visitor_home_cbgs'].fillna('')
    # remove empty rows
    df_part = df_part[(df_part['visitor_home_cbgs'] != '') & (df_part['visitor_home_cbgs'] != '{}')]
    # combine 30 files of the same month
    df_poi_monthly = pd.concat([df_poi_monthly, df_part], axis=0)
    # add month column
    df_poi_monthly['month'] = month
  return(df_poi_monthly)
# read the data for each month and combine
dfs={}
all_months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
dfs={}
for month in all_months:
  df_poi_monthly = single_month_raw_data(month)
  dfs[f'df_poi_{month}'] = df_poi_monthly

df_poi_raw = pd.concat(dfs.values(), ignore_index=True)


## assign brand tiers

In [18]:
# this data is produced by 01_brand_ratings.R
survey_data = pd.read_csv('clean_data/survey_ratings.csv')

In [19]:
survey_data = survey_data[~survey_data.brands.isin(['Nordstrom','Marshalls','Burlington'])]#remove department stores
# 100 most recognized brands
recognizable_brands = survey_data.sort_values('perc_na')[:100]

In [20]:
# sort by average ratings
recognizable_brands = recognizable_brands.sort_values(by='mean_rating',ascending=False)
# brand lists of 4 tiers
tier1_brands = recognizable_brands.iloc[0:25,]['brands'].values
tier2_brands = recognizable_brands.iloc[25:50,]['brands'].values
tier3_brands = recognizable_brands.iloc[50:75,]['brands'].values
tier4_brands = recognizable_brands.iloc[75:100,]['brands'].values

In [21]:
# assign tier to brands
def label_tier(brand):
  if pd.isnull(brand):
    return('no_brands')
  elif brand in tier1_brands:
    return('tier1')
  elif brand in tier2_brands:
    return('tier2')
  elif brand in tier3_brands:
    return('tier3')
  elif brand in tier4_brands:
    return('tier4')
  else: return('other_brands')

def preprocess_poi(df_poi):
  df_poi['brand_tier'] = df_poi.brands.apply(lambda x: label_tier(x))
  # remove poi with no brand recorded in the data
  df_poi = df_poi[df_poi.brand_tier != "no_brands"]
  df_poi['poi_cbg'] = df_poi['poi_cbg'].astype('str')
  # remove US outlying territories
  df_poi = df_poi.drop(df_poi[df_poi.poi_cbg.str.startswith(('60','66','69','78','72'))].index).reset_index(drop=True)
  return(df_poi)

In [23]:
# process poi data and add tiers
df_poi = preprocess_poi(df_poi_raw)

<ipython-input-21-e0b0d017b060>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poi['poi_cbg'] = df_poi['poi_cbg'].astype('str')


## county level visitors


we calculate 2 types of visitors:
1.   number of visitors who visited all clothing stores
2.   number of visitors who visited each brand tier.





### visitors for all clothing stores

In [27]:
def get_all_visitors(df_poi):
  # sgpy.unpack_json is a function used to explode JSON objects and calculate
  # number of visitors from census block groups
  df_home = sgpy.unpack_json(df_poi)
  # get county fips
  df_home['home_county'] = df_home.visitor_home_cbgs_key.str[:5]
  # calculate county level visitors
  df_county_home = df_home.groupby('home_county')['visitor_home_cbgs_value'].sum().reset_index()
  # rename
  df_county_home = df_county_home.rename(columns = {'visitor_home_cbgs_value':'all_visitors'})
  # calculate monthly average number of visitors
  df_county_home['all_visitors'] = df_county_home['all_visitors']/len(df_poi.month.unique())
  # drop counties in oversea territories and whose FIPS code starts with "CA" (wrong fips)
  df_county_home = df_county_home.drop(df_county_home[df_county_home.home_county.str.startswith(('60', '66', '69', '78', '72', 'CA'))].index)

  return(df_county_home)


In [28]:
df_all_visitors = get_all_visitors(df_poi)

### visitors for brand tiers

In [30]:
def get_tier_visitors(df_poi,tier):
    # Filter df_poi for the given tier
    df_poi_selected = df_poi[df_poi.brand_tier == tier]
    # Unpack json and prepare the dataframe
    df_home = sgpy.unpack_json(df_poi_selected)
    # get county fips
    df_home['home_county'] = df_home.visitor_home_cbgs_key.str[:5]
    # calculate county level visitors
    df_county_home = df_home.groupby('home_county')['visitor_home_cbgs_value'].sum().reset_index()
    df_county_home = df_county_home.rename(columns={'visitor_home_cbgs_value': 'tier_visitors'})
    # calculate monthly average number of visitors
    df_county_home['tier_visitors'] = df_county_home['tier_visitors']/len(df_poi_selected.month.unique()) #calculate average monthly visitors
    # Drop counties in oversea territories and whose FIPS code starts with "CA"
    df_county_home = df_county_home[~df_county_home.home_county.str.startswith(('60', '66', '69', '78', '72', 'CA'))].reset_index(drop=True)
    # add brand tier
    df_county_home['tier'] = tier
    return df_county_home

In [31]:
all_tiers = ['tier1','tier2','tier3','tier4']
dfs={}

for tier in all_tiers:
  df_home = get_tier_visitors(df_poi,tier)
  dfs[f"df_{tier}_visitors"] = df_home
df_tier_visitors = pd.concat(dfs.values(), ignore_index=True)
# long to wide data
df_tier_visitors_wide=df_tier_visitors.pivot(index='home_county',columns='tier',values="tier_visitors").fillna(0).reset_index()


In [43]:
# merge tier visitors with all visitors
df_visitors = df_all_visitors.merge(df_tier_visitors_wide,on="home_county",how='left').fillna(0)
# wide to long format
df_visitors_long = pd.melt(df_visitors, id_vars=['home_county','all_visitors'],
                           value_vars=['tier1','tier2','tier3','tier4'],
                           var_name='tier', value_name='tier_visitors')
# merging index will be used for merging with store accessibility data
df_visitors_long['merge_index'] = df_visitors_long.apply(lambda x: x.home_county+x.tier,axis=1)

## brand accessibility

This section calculates the percent of brand tier that are accessible to each home county. we expanded the polygon of each county by 20 miles beyond its original boundary.  Therefore, not only are these POIs located within the county boundary, but also those located near enough are considered accessible.

In [16]:
# Load the json file with county coordinates
geoData = gpd.read_file('https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson')


In [40]:
def poi_accessibility(df_poi):
  # select poi cordinates
  df_poi2=df_poi.loc[:,['latitude','longitude','brand_tier']]
  #transform points data to geodf format
  geodf_poi = gpd.GeoDataFrame(df_poi2, geometry=gpd.points_from_xy(df_poi2.longitude, df_poi2.latitude),crs='EPSG:4326')
  #transform points data to epsg27700 format (measured in meters)
  geodf_poi2=geodf_poi.to_crs('EPSG:27700')
  #expand county boundaries by 20 miles
  buffers_20mile = geoData.set_geometry(geoData.to_crs('EPSG:27700').buffer(32187))
  #merge county polygon and poi points
  dfmerge_county = gpd.sjoin(buffers_20mile, geodf_poi2, how="left", predicate='intersects')
  # calculate number of pois of different tiers within each county, "id" represents county code
  dfmerge_county = dfmerge_county[['id','brand_tier']].groupby(['id','brand_tier']).size().reset_index()
  dfmerge_county = dfmerge_county.rename(columns={0:'counts'})
  #average monthly counts, as df_poi includes 12 month data
  dfmerge_county['counts']=dfmerge_county['counts']/len(df_poi.month.unique())
  #count points of all brand type
  total_counts = dfmerge_county.groupby('id')['counts'].sum()
  # number of total accessible clothing stores
  dfmerge_county['total_counts'] = dfmerge_county.id.apply(lambda x: total_counts[x])
  # percent of accessible stores of given brand tier
  dfmerge_county['store_ratio'] = dfmerge_county.apply(lambda x: x.counts/x.total_counts,axis=1)
  # merging index that will be used for merging with visitor data
  dfmerge_county['merge_index'] = dfmerge_county.apply(lambda x: x.id+x.brand_tier,axis=1)

  return(dfmerge_county)


In [41]:
df_brand_accessibility = poi_accessibility(df_poi)

In [44]:
# merge visitor data with brand accessibility data
df_visitors_acc = df_visitors_long.merge(df_brand_accessibility[['merge_index','store_ratio']],
                                         on = 'merge_index',
                                         how='left').fillna(0)

In [ ]:
df_visitors_acc.to_csv("/clean_data/df_visits.csv",index=False)